In [147]:
from torch.utils.data import Dataset
from utilities import createAnnotation
import pandas as pd
from PIL import Image 

In [148]:
IMAGESROOTDIR = 'NINCO_OOD_classes'

In [149]:
class ImageDataset(Dataset):
    def __init__(self, rootDir):
        self.rootDir = rootDir
        createAnnotation(self.rootDir)
        self.annotation =  pd.read_csv('output.csv')


    def __getitem__(self, index):
        data_path = self.annotation.iloc[index,0]
        image = Image.open(data_path)
        label = self.annotation.iloc[index,1]
        return image, label

    def __len__(self):
        return len(self.annotation)

In [150]:
# instance of class ImageDataset
# contains all 765 images with their respective labels
imageDataset = ImageDataset(rootDir=IMAGESROOTDIR)

PermissionError: [Errno 13] Permission denied: 'output.csv'